In [1]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [2]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

In [4]:
import torch

# Set the device for training and querying the model.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The selected device is: "{DEVICE}"')

The selected device is: "cuda"


# Loading the Data

In [5]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'pems-bay')

In [6]:
import pickle
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [7]:
from src.spatial_temporal_gnn.model import SpatialTemporalGNN
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_pems_bay.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

# Get the STGNN and load the checkpoints.
spatial_temporal_gnn = SpatialTemporalGNN(9, 1, 12, 12, adj_matrix, DEVICE, 64)

stgnn_checkpoints_path = os.path.join('..', 'models', 'checkpoints',
                                      'st_gnn_pems_bay.pth')

stgnn_checkpoints = torch.load(stgnn_checkpoints_path)
spatial_temporal_gnn.load_state_dict(stgnn_checkpoints['model_state_dict'])

# Set the STGNN in evaluation mode.
spatial_temporal_gnn.eval();

In [8]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_pems_bay.csv'),
    has_header=False)

In [9]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [10]:
import pickle

# Get the data scaler.
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [11]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train.npy'))
y_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train.npy'))
x_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train_time.npy'))
y_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train_time.npy'))

x_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val.npy'))
y_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val.npy'))
x_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val_time.npy'))
y_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val_time.npy'))

x_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test.npy'))
x_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test_time.npy'))
y_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test_time.npy'))

In [12]:
from src.data.data_processing import get_distance_matrix

if not os.path.exists(
    os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy')):
    # Build the distance matrix between the nodes.
    distance_matrix = get_distance_matrix(
        adj_matrix,
        locations_df,
        node_pos_dict)

    # Save the distance matrix.
    np.save(os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'),
            distance_matrix)

else:
    # Load the distance matrix.
    distance_matrix = np.load(
        os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'))

In [13]:
from src.explanation.monte_carlo.explanation import apply_grid_search

apply_grid_search(
    x_train[::10],
    y_train[::10],
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts_list=[30, 50],
    explanation_size_factor_list=[2, 3],
    cut_size_factor_list=[2],
    exploration_weight_list=[5, 10, 20],
    remove_value_list=[0., -10.])

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: 0.0
[100/100] - 607s - MAE: { severe_congestion 11.2 -congestion 2.76 -free_flow 1.81 - total: 5.27 } - RMSE: { severe_congestion 11.5 -congestion 3.29 -free_flow 2.1 - total: 5.62 } - MAPE: { severe_congestion 56.1% -congestion 5.57% -free_flow 2.68% - total: 21.7% } - Average time: 6.07s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: -10.0
[100/100] - 607s - MAE: { severe_congestion 11.3 -congestion 2.56 -free_flow 1.97 - total: 5.29 } - RMSE: { severe_congestion 11.6 -congestion 2.99 -free_flow 2.19 - total: 5.61 } - MAPE: { severe_congestion 56.6% -congestion 5.18% -free_flow 2.92% - total: 21.8% } - Average time: 6.07s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 50 remove_value: 0.0
[100/100] - 996s - MAE: { severe_congestion 10.9 -congestion 2.33 -free_flow 1.69 - total

In [14]:
CUT_SIZE_FACTOR = 2
EXPLANATION_SIZE_FACTOR = 3
EXPLORATION_WEIGHT = 5
N_ROLLOUTS = 50

In [15]:
import os
import numpy as np

EXPLAINED_DATA_DIR = os.path.join(BASE_DATA_DIR, 'explained')
os.makedirs(EXPLAINED_DATA_DIR, exist_ok=True)


In [16]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the training set...')
x_train_explained, y_train_explained, train_scores = get_all_explanations(
    x_train,
    y_train,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train.npy'), x_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train.npy'), y_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'train_scores.npy'), train_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train_time.npy'), x_train_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train_time.npy'), y_train_time)


Computing the explanations for the training set...
[999/999] - 13738s - MAE: { severe_congestion 10.8 -congestion 1.95 -free_flow 1.46 - total: 4.73 } - RMSE: { severe_congestion 11.1 -congestion 2.37 -free_flow 1.64 - total: 5.04 } - MAPE: { severe_congestion 53.6% -congestion 3.9% -free_flow 2.16% - total: 19.9% } - Average time: 13.8s 


In [17]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the validation set...')
x_val_explained, y_val_explained, val_scores = get_all_explanations(
    x_val,
    y_val,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val.npy'), x_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val.npy'), y_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'val_scores.npy'), val_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val_time.npy'), x_val_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val_time.npy'), y_val_time)

Computing the explanations for the validation set...
[198/198] - 2829s - MAE: { severe_congestion 10.5 -congestion 2.03 -free_flow 1.41 - total: 4.57 } - RMSE: { severe_congestion 10.9 -congestion 2.46 -free_flow 1.63 - total: 4.89 } - MAPE: { severe_congestion 52.6% -congestion 4.22% -free_flow 2.11% - total: 19.3% } - Average time: 14.3s 


In [18]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the test set...')
x_test_explained, y_test_explained, test_scores = get_all_explanations(
    x_test,
    y_test,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test.npy'), x_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test.npy'), y_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'test_scores.npy'), test_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test_time.npy'), x_test_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test_time.npy'), y_test_time)

Computing the explanations for the test set...
[300/300] - 4006s - MAE: { severe_congestion 10.1 -congestion 2.04 -free_flow 1.47 - total: 4.52 } - RMSE: { severe_congestion 10.4 -congestion 2.46 -free_flow 1.65 - total: 4.81 } - MAPE: { severe_congestion 49.2% -congestion 4.08% -free_flow 2.19% - total: 18.4% } - Average time: 13.4s 
